In [1]:
from gurobipy import*
import math

In [2]:
p = {
0:[20,1],
1: [30,4],
2: [50,10]}
x = [1,5]
dist = {}
for key in p:
    dist["x",key] = math.floor(math.sqrt(pow(x[0] - p[key][0],2) + pow(x[1] - p[key][1],2))*100)/100

# 最大初速
V = 3
dist

{('x', 0): 19.41, ('x', 1): 29.01, ('x', 2): 49.25}

In [3]:
model = Model("Sheep")
x = {}
# cost variable
c = {}
m= {}
t = 40
M = 99999
for i in p:
    for j in range(t):
        x[i, j] = model.addVar(vtype = GRB.BINARY, name = "x{},{}".format(i,j))
        c[i,j] = model.addVar(vtype = GRB.INTEGER, name = "c{},{}".format(i,j))
for i in range(t):
    m[i] = model.addVar(vtype = GRB.BINARY, name = "m{}".format(i))
# cost constraints
for i in p:
    for j in range(t):
        model.addConstr(V*x[i,j]*m[j] <= c[i,j], name =  "(1)")
        model.addConstr(M*(1-x[i,j]) + V*x[i,j]*m[j] >= c[i,j], name = "(2)")
        for k in range(max(0, j-V+1), j):  
            model.addConstr((V-(j-k))*x[i,k]*quicksum(x[k,j] for k in p) <= c[i,j], name = "(1)")
            model.addConstr(quicksum(x[i,n] for n in range(k+1, j+1))*M + M*(1-x[i,k]) +  (V-(j-k)) *x[i,k]*m[j] >= c[i,j], name = "(2)")
        model.addConstr(0 <= c[i,j], name = "(3)")
        model.addConstr(quicksum(x[i,k] for k in range(max(0, j-V+1), j+1))*M >= c[i,j], name = "(3)" )
#         model.addConstr(M*quicksum(x[k,j] for k in p) >= c[i,j], name = "(test)" )
        

# distance constraints
for i in p:
#     lhs = LinExpr(0)
#     for j in range(t):
#         lhs += x[i,j]*c[i,j]
#         if(j >= 1):
#             lhs += x[i,j-1]*c[i,j]
#         if(j >= 2):
#             lhs += x[i,j-2]*c[i,j]
#     model.addConstr(lhs >= dist['x', i], name = "(3)")
    model.addConstr(quicksum(c[i,j] for j in range(t)) >= dist['x', i], name = "(4)")
    
# 無人機至少停留在　a , b, c　其中一點（或沒有）
for j in range(t):
    model.addConstr(quicksum(x[i,j] for i in p) <= 1, name = "(5)")
    model.addConstr(quicksum(x[i,j] for i in p) >= m[j], name = "(7)")
# t-1 沒有停留時，ｔ也會＝０
for j in range(1,t):
    model.addConstr(quicksum(x[i,j] for i in p) <= quicksum(x[i,j-1] for i in p), name= "(6)")

# non-negative
for i in p:
    for j in range(t):
        model.addConstr(x[i,j] >= 0, name="(8)")



Academic license - for non-commercial use only - expires 2021-08-28
Using license file C:\Users\linda\gurobi.lic


In [4]:
# objective
lhs = LinExpr(0)
for i in p:
#     for j in range(t):
#         lhs += x[i,j]*c[i,j]
#         if(j >= 1):
#             lhs+= x[i,j-1]*c[i,j]
#         if(j >= 2):
#             lhs += x[i,j-2]*c[i,j]
    lhs += quicksum(c[i,j] for j in range(t))
# minimize time spend
lhs_t = LinExpr(quicksum(x[i,j] for i in p for j in range(t)))
model.setObjective(lhs_t,GRB.MINIMIZE)
model.update()
# model.display()
model.optimize()
print("Optimize value = ", model.ObjVal)


Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 482 rows, 280 columns and 1345 nonzeros
Model fingerprint: 0x4d243ccc
Model has 702 quadratic constraints
Variable types: 0 continuous, 280 integer (160 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  QMatrix range    [1e+00, 3e+00]
  QLMatrix range   [1e+00, 1e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
  QRHS range       [1e+05, 1e+05]
Presolve removed 10 rows and 0 columns
Presolve time: 0.05s
Presolved: 1765 rows, 631 columns, 5655 nonzeros
Variable types: 0 continuous, 631 integer (511 binary)

Root relaxation: objective 5.882353e+00, 288 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    5.88235    0   68    

In [95]:
# model.display()

In [5]:
ans = {}
for var in model.getVars(): 
    if(var.varName[0] == 'x' and var.x > 0.01):
        print('{name} : {val}'.format(name=var.varName, val = var.x))
        key = var.varName.split(',')
        ans[key[1]] = key[0][1:]
#     if(var.varName[0] == 'c' and var.x > 0):
#         print('{name} : {val}'.format(name=var.varName, val = var.x))
list = [None]*len(ans)
for key in ans:
    list[int(key)] = int(ans[key])+1

x0,2 : 1.0
x0,6 : 1.0
x0,15 : 1.0
x0,19 : 1.0
x1,1 : 1.0
x1,4 : 1.0
x1,8 : 1.0
x1,12 : 1.0
x1,17 : 1.0
x2,0 : 1.0
x2,3 : 1.0
x2,5 : 1.0
x2,7 : 1.0
x2,9 : 1.0
x2,10 : 1.0
x2,11 : 1.0
x2,13 : 1.0
x2,14 : 1.0
x2,16 : 1.0
x2,18 : 1.0


In [6]:
def totalS(V):
    total = 0
    for i in range(1,V+1):
        total+=i
    return total
def score(list,n,V,dis):
    scores = 0
    penalty = 0
    flag = False
    for j in range(1,n+1):
#        move.clear();
        curr_score = 0
        for i in range(len(list)):
            flag = False
            if(list[i] == j):
#                 // move.push_back(V);
                curr_score += V
            else:
#                 // max{0, i-V+1} ... i - 1
                index = 0
                if(i - V + 1 > index):
                    index = i - V + 1
                k = i - 1
                while(k >= index):
                    if(list[k] == j):
                        flag = True
#                         // move.push_back((V-(i-k)))
                        curr_score += V-(i-k)
                        break
                    k -= 1
#         // 補償t = list.size() 後沒加上的距離
#         i = len(list)-1
# #         print(i)
#         while( i >= len(list) - V):
#             if(list[i] == j):
#                 curr_score += totalS(V- (len(list) - i))
#                 break
#             i-=1
        scores += curr_score
        print(curr_score)
        if(curr_score - dis[j-1] < 0):
            penalty += curr_score - dis[j-1]
        else:
            penalty += 1
        
    if(penalty == 3):
#         // 都有達到目標
        penalty = 10
    return scores, penalty

In [7]:
# list = [2,2,2,3,3,3,3,1,3,1,2]
dis = [9.84, 14.03, 19.64]
scores, penalty = score(list,len(p),V,dis)
print(list)
print(scores)
print(penalty)
# [2,2,3,1,3]

21
30
50
[3, 2, 1, 3, 2, 3, 1, 3, 2, 3, 3, 3, 2, 3, 3, 1, 3, 2, 3, 1]
101
10
